# Phase 4: Train Random Forest Classifier

## Objectives
1. Train Random Forest on manually labeled data
2. Time-based train/test split
3. Start with 2-3 classes: Healthy, Cloudy, Investigate
4. Evaluate confidence calibration

In [ ]:
import sys
sys.path.insert(0, '/workspaces/O-M-Monte-Carlo')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

from src.phase4_model.rf_classifier import SolarAnomalyRF

sns.set_style('darkgrid')
print("Imports successful!")

## Load Labeled Data